In [1]:
%load_ext autoreload
%autoreload 2

In [211]:
import pyphi
import pickle
import numpy as np
import string
import plotly.graph_objects as go
import math
pi = math.pi

def strip_punct(s):
    return str(
        s.translate(str.maketrans({key: None for key in string.punctuation})).replace(
            " ", ""
        )
    )

def find_continuous_members(members):
    continuous_members = [m for m in members
        if m==tuple(range(m[0],m[0]+len(m)))
    ]
    return continuous_members

def circlepoints(r,n):
    return [(math.cos(2*pi/n*x)*r,math.sin(2*pi/n*x)*r) for x in range(0,n)]

def divide_mechs_by_order(mechs):
    top_order = max([len(m) for m in mechs])
    return [[m for m in mechs if len(m)==o] for o in range(1,top_order+1)]

def make_continuous_mech_cone(mechs,radii=None,layer_height=1):
    continuous_mechs = find_continuous_members(mechs)
    mechs_by_order = divide_mechs_by_order(continuous_mechs)
    orders_n = len(mechs_by_order)
    if radii is None:
        radii = [len(order)-1 for order in mechs_by_order]
    coords = []
    zs = range(0,orders_n*layer_height,layer_height)
    for o,order in enumerate(mechs_by_order):
        positions = circlepoints(radii[o],len(order))
        order_coords = np.array([[position[0],position[1],zs[o]] for position in positions])
        coords.append(order_coords)
        
    return coords

In [233]:
subsystem = pickle.load(open('space_subsystem.pkl','rb'))
ces = pickle.load(open('space_ces_with_nodelabels.pkl','rb'))
mechs = [c.mechanism for c in ces]

coords = make_continuous_mech_cone(mechs, radii=[4,5,6,6,5,4,3,0], layer_height=2)

continuous_mechs = find_continuous_members(mechs)

labels_by_order = [[strip_punct(str(subsystem.indices2nodes(m))) for m in mechs] for mechs in divide_mechs_by_order(continuous_mechs)]

In [234]:
# traces = [
#     go.Scatter3d(x=coords[o][:,0],y=coords[o][:,1],z=coords[o][:,2], mode='markers+text',text=labels_by_order[o])
#     for o in range(len(coords))
# ]

# fig = go.Figure(traces)
# # fig.update_layout(height=1000,width=1400)
# fig.show()
# # fig.write_html("space-positions.html")

# Other stuff

In [235]:
import pyphi.visualize as vis
import random

In [236]:
continuous_3o_mechs = [m for m in continuous_mechs if len(m)==3]

In [237]:
allrelations = pickle.load(open('space_2relations.pkl','rb'))
relations5k = random.sample(allrelations,5000)
relations15k = random.sample(allrelations,15000)
example_mechs = [(3,4),(2,3,4),(3,4,5),(2,3,4,5)]
cde_def_rels = [r for r in allrelations if (2,3,4) in r.mechanisms or (3,4,5) in r.mechanisms]
cde_rels = [r for r in allrelations if (2,3,4) in r.mechanisms]
example_rels = [r for r in allrelations if any(m in r.mechanisms for m in example_mechs)]
third_order_contexts = [r for r in allrelations if any(cm in r.mechanisms for cm in continuous_3o_mechs)]
continuous_mechs_rels = [r for r in allrelations if all(m in continuous_mechs for m in r.mechanisms)]
relations = third_order_contexts

In [238]:
continuous_mechs_coords = np.concatenate([coord for coord in coords])

In [239]:
discontinuous_mechs_z = 10

mech_coords = []
purv_coords = []

for c in ces:
    if c.mechanism in continuous_mechs:
        for m,mech in enumerate(continuous_mechs):
            if c.mechanism==mech:
                mech_coords.append(continuous_mechs_coords[m])
                purv_coords.append(continuous_mechs_coords[m])
                purv_coords.append(continuous_mechs_coords[m])
    else:
        mech_coords.append(np.array([0,0,discontinuous_mechs_z]))
        purv_coords.append(np.array([0,0,discontinuous_mechs_z]))
        purv_coords.append(np.array([0,0,discontinuous_mechs_z]))        

mech_coords = np.squeeze(mech_coords)
purv_coords = np.squeeze(purv_coords)

mech_coords = mech_coords.transpose()

In [240]:
# fig = vis.plot_ces_epicycles(subsystem,ces,relations,network_name='space-cone-3rd-o-contexts',
#     link_width_range=(1, 3),
#     eye_coordinates=(-1.5, 1, 0),
#     mechanism_labels_size=15,
#     purview_labels_size=8,
#     mechanism_label_position="middle center",
#     purview_label_position="middle center",                         
#     user_mechanism_coords=(mech_coords),
#     user_purview_coords=purv_coords,
#     hovermode=False,
#     show_links=False,
#     show_purview_labels='legendonly',
#     edge_size_range=(1, 5),
#                             )

In [244]:
fig = vis.plot_ces_epicycles(subsystem,ces,example_rels,network_name='space-cone-example-mechs',
    link_width_range=(1, 3),
    eye_coordinates=(-1.5, 1, 0),
    mechanism_labels_size=15,
    purview_labels_size=8,
    mechanism_label_position="middle center",
    purview_label_position="middle center",                         
    user_mechanism_coords=(mech_coords),
    user_purview_coords=purv_coords,
    hovermode=False,
    show_links=False,
    show_purview_labels='legendonly',
    edge_size_range=(0.1, 5),
                            )

Computing edges:   0%|          | 0/1808 [00:00<?, ?it/s]

In [247]:
# fig = vis.plot_ces_epicycles(subsystem,ces,example_rels,network_name='space-cone-example-mechs-intersect',
#     link_width_range=(1, 3),
#     eye_coordinates=(-1.5, 1, 0),
#     mechanism_labels_size=15,
#     purview_labels_size=8,
#     mechanism_label_position="middle center",
#     purview_label_position="middle center",                         
#     user_mechanism_coords=(mech_coords),
#     user_purview_coords=purv_coords,
#     hovermode=False,
#     show_links=False,
#     show_purview_labels='legendonly',
#     edge_size_range=(0.1, 5),
#     intersect_mechanisms=example_mechs
#                             )

Computing edges:   0%|          | 0/1808 [00:00<?, ?it/s]

In [243]:
fig = vis.plot_ces_epicycles(subsystem,ces,continuous_mechs_rels,network_name='space-cone-continuous-mechs-rels',
    link_width_range=(1, 3),
    eye_coordinates=(-1.5, 1, 0),
    mechanism_labels_size=15,
    purview_labels_size=8,
    mechanism_label_position="middle center",
    purview_label_position="middle center",                         
    user_mechanism_coords=(mech_coords),
    user_purview_coords=purv_coords,
    hovermode=False,
    show_links=False,
    show_purview_labels='legendonly',
    edge_size_range=(0.01, 7),
#     show_edges=False,
                             
                            )

Computing edges:   0%|          | 0/1716 [00:00<?, ?it/s]

In [248]:
fig = vis.plot_ces_epicycles(subsystem,ces,cde_def_rels,network_name='space-cone-cde-def',
    link_width_range=(1, 3),
    eye_coordinates=(-1.5, 1, 0),
    mechanism_labels_size=15,
    purview_labels_size=8,
    mechanism_label_position="middle center",
    purview_label_position="middle center",                         
    user_mechanism_coords=(mech_coords),
    user_purview_coords=purv_coords,
    hovermode=False,
    show_links=False,
    show_purview_labels='legendonly',
    edge_size_range=(1, 5),
                            )

Computing edges:   0%|          | 0/912 [00:00<?, ?it/s]

In [209]:
# fig = vis.plot_ces_epicycles(subsystem,ces,relations15k,network_name='space-cone-relations15k',
#     link_width_range=(1, 3),
#     eye_coordinates=(-1.5, 1, 0),
#     mechanism_labels_size=15,
#     purview_labels_size=8,
#     mechanism_label_position="top center",
#     purview_label_position="middle center",                         
#     user_mechanism_coords=(mech_coords),
#     user_purview_coords=purv_coords,
#     hovermode=False,
#     show_links=False,
#     show_purview_labels='legendonly',
# #     edge_size_range=(1, 5),
#                             )

In [210]:
# fig = vis.plot_ces_epicycles(subsystem,ces,allrelations,network_name='space-cone-all2relations',
#     link_width_range=(1, 3),
#     eye_coordinates=(-1.5, 1, 0),
#     mechanism_labels_size=15,
#     purview_labels_size=8,
#     mechanism_label_position="middle center",
#     purview_label_position="middle center",                         
#     user_mechanism_coords=(mech_coords),
#     user_purview_coords=purv_coords,
#     hovermode=False,
#     show_links=False,
#     show_purview_labels='legendonly',
# #     edge_size_range=(1, 5),
#                             )